# Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

оценка 2 и ниже — низкий рейтинг;

оценка 4 и ниже — средний рейтинг;

оценка 4.5 и 5 — высокий рейтинг.

Результат классификации запишите в столбец class.

In [ ]:
import pandas as pd

def get_category(ratings_group):
    r_mean = ratings_group.rating.mean()
    
    if r_mean <= 2:
        return 'низкий рейтинг'
    elif r_mean <= 4:
        return'средний рейтинг'
    else:
        return 'высокий рейтинг'

    
def get_movies_with_class(movies, ratings):
    '''
    Добавляет к фрейму movies колонку class c категорией среднего рейтинга 
    на основе данных из фрейма ratings 
    '''
    means = ratings.groupby('movieId').apply(get_category)
    means.name = "class"
    return movies.set_index('movieId').join(means).reset_index()


movies = pd.read_csv(r'ml-latest-small\movies.csv')
ratings = pd.read_csv(r'ml-latest-small\ratings.csv')

get_movies_with_class(movies, ratings)

# Задание 2

Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [ ]:
import pandas as pd

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

def try_get_region(keyword):
    '''
    Пробуем получить регион по вхождению города в поисковый запрос.
    Только на основе определенных городов в geo_data.
    '''
    for key, values in geo_data.items():
        for value in values:
            if value in keyword:
                return key
            
    return 'undefined'
            

keywords = pd.read_csv(r'ml-latest-small\keywords.csv')

keywords['region'] = keywords.keyword.apply(try_get_region)

assert try_get_region('владивосток') == 'Дальний Восток'
assert try_get_region('аа,псков ррр дд ') == 'Северо-Запад'
assert try_get_region('') == 'undefined'
assert try_get_region('Что то ороро') == 'undefined'

# Задание 3

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010 года.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:<br>

    2.1 для каждой строки пройдите по всем годам списка years;<br>
    
    2.2 если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;<br>
    
    2.3 если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.<br>


3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

In [ ]:
import pandas as pd

# (тут актуализирую даты из задания, для актуальной картины!!!)
years = [str(year) for year in range(1950, 2024)]

def production_year(title):
    '''
    Определяем год (в пределах с 1950 по 2023) из заголовка
    В случае, если не смогли определеить - возвращаем 1900
    '''
    for year in years:
        if year in title:
            return year
        
    return '1900'

movies = pd.read_csv(r'ml-25m\movies.csv')
movies['year'] = movies.title.apply(production_year)


ratings = pd.read_csv(r'ml-latest-small\ratings.csv')

statistics = (ratings.set_index('movieId')
    .join(movies.set_index('movieId'))
    .groupby('year')
    .apply(lambda group: group.rating.mean())
    .sort_values(ascending=False)
    .rename('rating', inplace=True)
    .reset_index())

statistics